In [1]:
import pandas as pd
import time
import numpy as np
from tqdm import tqdm
from PIL import Image
import jittor as jt
import jittor.nn as nn
import jittor.optim as optim
from jittor.dataset import Dataset
from jittor.dataset import DataLoader
from jittor import transform
import os
import matplotlib.pyplot as plt
import jclip as clip

train_label = pd.read_csv('/mnt/workspace/JCLIP/Dataset/train_1234.txt')
val_label = pd.read_csv('/mnt/workspace/JCLIP/Dataset/valid.txt')
test_label = pd.read_csv('/mnt/workspace/JCLIP/Dataset/test.txt')

[i 0805 17:36:26.278341 92 lock.py:85] Create lock file:/root/.cache/jittor/jt1.3.9/g++11.4.0/py3.10.14/Linux-4.19.24-x10/IntelRXeonRPlaxf9/d731/jittor.lock
[i 0805 17:36:26.286400 92 compiler.py:956] Jittor(1.3.9.5) src: /usr/local/lib/python3.10/site-packages/jittor
[i 0805 17:36:26.288514 92 compiler.py:957] g++ at /usr/bin/g++(11.4.0)
[i 0805 17:36:26.288587 92 compiler.py:958] cache_path: /root/.cache/jittor/jt1.3.9/g++11.4.0/py3.10.14/Linux-4.19.24-x10/IntelRXeonRPlaxf9/d731/default
[i 0805 17:36:26.300494 92 __init__.py:412] Found nvcc(12.1.105) at /usr/local/cuda/bin/nvcc.
[i 0805 17:36:26.485802 92 __init__.py:412] Found gdb(22.04.2) at /usr/bin/gdb.
[i 0805 17:36:26.492218 92 __init__.py:412] Found addr2line(2.38) at /usr/bin/addr2line.
[i 0805 17:36:26.909003 92 compiler.py:1011] cuda key:cu12.1.105_sm_70
[i 0805 17:36:26.936283 92 compiler.py:34] Create cache dir: /root/.cache/jittor/jt1.3.9/g++11.4.0/py3.10.14/Linux-4.19.24-x10/IntelRXeonRPlaxf9/d731/default/cu12.1.105_sm_

[i 0805 17:36:33.827751 80 log.cc:351] Load log_sync: 1
[i 0805 17:36:49.100478 80 __init__.py:227] Total mem: 31.41GB, using 10 procs for compiling.


[i 0805 17:37:49.221617 80 log.cc:351] Load log_sync: 1
[i 0805 17:37:49.222705 80 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0805 17:37:49.351494 80 init.cc:63] Found cuda archs: [70,]


Compiling jittor_core(151/151) used: 59.729s eta: 0.000s


[i 0805 17:37:51.972347 80 __init__.py:412] Found mpicc(4.1.2) at /usr/bin/mpicc.
[i 0805 17:37:51.986403 80 compiler.py:34] Create cache dir: /root/.cache/jittor/jt1.3.9/g++11.4.0/py3.10.14/Linux-4.19.24-x10/IntelRXeonRPlaxf9/d731/default/cu12.1.105_sm_70/custom_ops


[i 0805 17:37:54.619926 80 compile_extern.py:378] Downloading cutt...
[i 0805 17:37:54.631207 80 compile_extern.py:391] installing cutt...


Compiling jittor_mpi_core(7/7) used: 2.524s eta: 0.000s
Compiling libcutt(9/9) used: 12.856s eta: 0.000s


[i 0805 17:38:13.332327 80 compiler.py:34] Create cache dir: /root/.cache/jittor/jt1.3.9/g++11.4.0/py3.10.14/Linux-4.19.24-x10/IntelRXeonRPlaxf9/d731/default/cu12.1.105_sm_70/cuda


Compiling gen_ops_mkl_test_mkl_conv_mkl_matmul_mkl_conv_back___hashcad247(7/7) used: 3.839s eta: 0.000s
Compiling gen_ops_cub_test_cub_cumsum_cub_argsort_cub_where____hash4eb589(6/6) used: 3.039s eta: 0.000s
Compiling gen_ops_cublas_test_cublas_acc_matmul_cublas_matmu___hash02a226(8/8) used: 2.529s eta: 0.000s
Compiling gen_ops_cudnn_conv_backward_w_cudnn_conv3d_backwar___hash25ad86(16/16) used: 6.876s eta: 0.000s


In [2]:
train_label['path'] = '/mnt/workspace/JCLIP/Dataset/' + train_label['img_name']
val_label['path'] = '/mnt/workspace/JCLIP/Dataset/' + val_label['img_name']
test_label['path'] = '/mnt/workspace/JCLIP/Dataset/TestSetA/' + test_label['img_name']

In [3]:
val_label

,img_name,target,path
0,TrainSet/Animal/Bear/5.jpg,0,/mnt/workspace/JCLIP/Dataset/TrainSet/Animal/B...
1,TrainSet/Animal/Bear/6.jpg,0,/mnt/workspace/JCLIP/Dataset/TrainSet/Animal/B...
2,TrainSet/Animal/Bear/7.jpg,0,/mnt/workspace/JCLIP/Dataset/TrainSet/Animal/B...
3,TrainSet/Animal/Bear/8.jpg,0,/mnt/workspace/JCLIP/Dataset/TrainSet/Animal/B...
4,TrainSet/Animal/Bear/9.jpg,0,/mnt/workspace/JCLIP/Dataset/TrainSet/Animal/B...
...,...,...,...
3735,TrainSet/Thu-dog/papillon/10.jpg,373,/mnt/workspace/JCLIP/Dataset/TrainSet/Thu-dog/...
3736,TrainSet/Thu-dog/papillon/11.jpg,373,/mnt/workspace/JCLIP/Dataset/TrainSet/Thu-dog/...
3737,TrainSet/Thu-dog/papillon/12.jpg,373,/mnt/workspace/JCLIP/Dataset/TrainSet/Thu-dog/...
3738,TrainSet/Thu-dog/papillon/13.jpg,373,/mnt/workspace/JCLIP/Dataset/TrainSet/Thu-dog/...


In [4]:
test_label['img_name'][0]

'image_855.jpeg'

# 模型训练与验证

In [8]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = ""

    def pr2int(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [9]:
def validate(val_loader, model, criterion, bs_value):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    batch_num = int(len(val_loader) / bs_value)
    progress = ProgressMeter(batch_num, batch_time, losses, top1)

    # switch to evaluate mode
    model.eval()

    # 加载clip
    model_clip, preprocess = clip.load("/mnt/workspace/JCLIP/ViT-B-32.pkl")

    with jt.no_grad():
        end = time.time()
        for i, (imgs, target) in tqdm(enumerate(val_loader), total=batch_num):
            # 将 NumPy 数组转换为 PIL 图像并进行预处理
            images = [Image.fromarray(img_np) for img_np in imgs.numpy()]

            # 预处理图像
            images_preprocessed = jt.cat([preprocess(image).unsqueeze(0) for image in images], dim=0)
            images_preprocessed = jt.misc.to(images_preprocessed, 'cuda')

            images_clip_features = model_clip.encode_image(images_preprocessed)
            # [batch_size, 512]
            images_clip_features /= images_clip_features.norm(dim=-1, keepdim=True)

            target = jt.misc.to(target, 'cuda')

            # compute output
            output = model(images_clip_features)

            loss = criterion(output, target)

            # measure accuracy and record loss
            acc = (output.argmax(1)[0].view(-1) == target.float().view(-1)).float().mean() * 100

            losses.update(loss.item(), imgs.size(0))
            top1.update(acc, imgs.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f}'
              .format(top1=top1))
        return top1


def predict(test_label, model, tta=10):
    # switch to evaluate mode
    model.eval()

    # 加载clip
    model_clip, preprocess = clip.load("/mnt/workspace/JCLIP/ViT-B-32.pkl")

    test_num = len(test_label)

    test_pred_tta = None
    for _ in range(tta):
        test_pred = []
        with jt.no_grad():
            end = time.time()
            for i in tqdm(range(test_num), total=test_num):

                img_path = test_label['path'][i]
                image = Image.open(img_path)
                image = preprocess(image).unsqueeze(0)

                image_features = model_clip.encode_image(image)
                image_features /= image_features.norm(dim=-1, keepdim=True)

                # compute output
                output = model(image_features)

                output = nn.softmax(output, dim=1)
                output = output.numpy()

                test_pred.append(output)
        test_pred = np.vstack(test_pred)

        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred

    return test_pred_tta


def train(train_loader, model, criterion, optimizer, epoch, bs_value):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')

    # jittor的len(train_loader)不会随着batch_size的变化而改变
    # 永远都是1496
    batch_num = int(len(train_loader) / bs_value)
    progress = ProgressMeter(batch_num, batch_time, losses, top1)

    # switch to train mode
    model.train()

    # 加载clip
    model_clip, preprocess = clip.load("/mnt/workspace/JCLIP/ViT-B-32.pkl")
    model_clip.cuda()

    end = time.time()
    for i, (imgs, target) in enumerate(train_loader):
        # img [1, 224, 224, 3]  但是用不着

        # 将 NumPy 数组转换为 PIL 图像并进行预处理
        images = [Image.fromarray(img_np) for img_np in imgs.numpy()]


        # 预处理图像
        images_preprocessed = jt.cat([preprocess(image).unsqueeze(0) for image in images], dim=0)
        images_preprocessed = jt.misc.to(images_preprocessed, 'cuda')

        images_clip_features = model_clip.encode_image(images_preprocessed)
        # [batch_size, 512]
        images_clip_features /= images_clip_features.norm(dim=-1, keepdim=True)

        target = jt.misc.to(target, 'cuda')

        # compute output
        output = model(images_clip_features)
        # output [batch_size, 374] target[batch_size]

        loss = criterion(output, target)

        # measure accuracy and record loss
        losses.update(loss.item(), imgs.size(0))

        # print(f'tput.argmax(1)：{output.argmax(1)}')
        # print(f'tput.argmax(1)[0]的shape：{output.argmax(1)[0].shape}')
        # tput.argmax(1)：(jt.Var([351 351 285 131], dtype=int32), jt.Var([0.0555831  0.0470255  0.04495085 0.04521766], dtype=float32))
        # 所以要改为output.argmax(1)[0]
        acc = (output.argmax(1)[0].view(-1) == target.float().view(-1)).float().mean() * 100
        top1.update(acc, n=imgs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        optimizer.backward(loss)
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 4 == 0:
            progress.pr2int(i)

# 数据集

In [10]:
class FFDIDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        super().__init__()
        self.img_path = img_path
        self.img_label = img_label

        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')
        label = self.img_label[index]

        if self.transform is not None:
            img = self.transform(img)
        # jt.array(np.array(self.img_label[index]), dtype=np.float32)
#         label = jt.array(np.array(self.img_label[index]), dtype=np.float32)

#         print(f'label处理前{self.img_label[index]}')
#         print(f'label处理后{label}')
#         print(f'img类型 和 形状：{img.dtype} {img.shape}')
#         print(f'label类型 和 形状：{label.dtype} {label.shape}')

        # 返回必须为图片和int，不要去变成tensor
        return img, label

    def __len__(self):
        return len(self.img_path)

# 模型定义

In [11]:


class eca_layer(nn.Module):
    def __init__(self, channels, k_size=3):
        super(eca_layer).__init__()
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False) 
        self.sigmoid = nn.Sigmoid()

    def execute(self, x):

        original_x = x
        # Two different branches of ECA module
        x = self.conv(x.unsqueeze(-1).transpose(-1, -2)).transpose(-1, -2).squeeze(-1)
        # Multi-scale information fusion
        x = self.sigmoid(x)
        return original_x * x

class attention_classifier(nn.Module):
    def __init__(self, in_channels, out_channels, textfeatures_weights):
        super(attention_classifier).__init__()
        self.fc1 = nn.Linear(in_features=in_channels, out_features=out_channels, bias=False)
        self.eca_layer = eca_layer(in_channels)
        self.init_weights(textfeatures_weights)

    def init_weights(self, textfeatures_weights):
        # 初始化权重
        self.fc1.weight = jt.array(textfeatures_weights)

    def execute(self, x):

        # 通道注意力
        x = self.eca_layer(x)

        x = self.fc1(x)

        return x

# class attention_classifier(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(attention_classifier).__init__()
#         self.fc1 = nn.Sequential(
#             nn.Linear(in_channels, 768, bias=False),
#             nn.BatchNorm1d(768),
#             nn.ReLU(),

#             nn.Dropout(),
#             nn.Linear(768, out_channels, bias=False),
#         )

#     def execute(self, x):
#         x = self.fc1(x)
#         return x

In [12]:
def process_textfeatures():
    # 加载clip
    model_clip, preprocess = clip.load("/mnt/workspace/JCLIP/models_pkl/clip_ft/clip_ft_Epoch47_100.0.pkl")
    model_clip.cuda()


    classes = open('/mnt/workspace/JCLIP/Dataset/classes.txt').read().splitlines()
    # remove the prefix Animal, Thu-dog, Caltech-101, Food-101

    new_classes1 = []
    new_classes2 = []
    for c in classes:
        c = c.split(' ')[0]
        if c.startswith('Animal'):
            c = c[7:]
            c1 = 'A photo of ' + c.lower().replace('_', ' ') + ', a type of animal'
            c2 = 'A photo animal of ' + c.lower().replace('_', ' ') #+ '.'
        if c.startswith('Thu-dog'):
            c = c[8:]
            c1 = 'A photo of ' + c.lower().replace('_', ' ') + ', a type of dog'
            c2 = 'A photo dog of ' + c.lower().replace('_', ' ') #+ '.'
        if c.startswith('Caltech-101'):
            c = c[12:]
            c1 = 'A photo of ' + c.lower().replace('_', ' ')
            c2 = 'A photo of ' + c.lower().replace('_', ' ') #+ '.'
        if c.startswith('Food-101'):
            c = c[9:]
            c1 = 'A photo of ' + c.lower().replace('_', ' ') + ', a type of food'
            c2 = 'A photo food of ' + c.lower().replace('_', ' ') #+ '.'
        new_classes1.append(c1)
        new_classes2.append(c2)

    text1 = clip.tokenize(new_classes1)
    text2 = clip.tokenize(new_classes2)


    text_features1 = model_clip.encode_text(text1)
    text_features1 /= text_features1.norm(dim=-1, keepdim=True)

    text_features2 = model_clip.encode_text(text2)
    text_features2 /= text_features2.norm(dim=-1, keepdim=True)

    text_features = (text_features1 + text_features2) / 2.
    text_features /= text_features.norm(dim=-1, keepdim=True)
    return text_features

In [13]:
text_features = process_textfeatures()

[i 0805 17:49:11.800391 80 cuda_flags.cc:49] CUDA enabled.


In [16]:
# parameter
epoch_num = 1
bs_value = 4

In [14]:
model_path = "../models_pkl/model_linear/model+compress_Epoch.pkl"

model = attention_classifier(in_channels=512, out_channels=374, textfeatures_weights=text_features)


if os.path.exists(model_path):
    model.load_state_dict(jt.load(model_path))
    print("使用以往的模型继续训练")
else:
    print("使用hugging face预训练模型")


model = model.cuda()

使用hugging face预训练模型


In [17]:
train_trans = transform.Compose([
                transform.Resize((224, 224)),
                transform.RandomRotation(degrees=180),
                transform.ColorJitter(0.5),
                transform.RandomHorizontalFlip(),
                transform.RandomVerticalFlip(),
                # transform.ToTensor(),
                # transform.ImageNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
val_trans = transform.Compose([
                transform.Resize((224, 224)),
                # transform.ToTensor(),
                # transform.ImageNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
train_dataset = FFDIDataset(train_label['path'], train_label['target'], train_trans).set_attrs(batch_size=bs_value, shuffle=True)
val_dataset = FFDIDataset(val_label['path'], val_label['target'], val_trans).set_attrs(batch_size=bs_value, shuffle=False)
train_loader = DataLoader(train_dataset)
val_loader = DataLoader(val_dataset)

In [30]:
# print(train_loader)
# print(len(train_loader))
# for i, (img, target) in enumerate(train_loader):
#     # img_array = img.numpy()
#     # print(img_array.shape)
#     # images_features = process_images_to_features(img_array)
#     # print(images_features.shape)
#     # print(type(images_features))
#     # print(target.shape)
#     # print(type(target))
#     if i == 0:
#         print(type(img))
#         # print(img.transpose((0, 3, 1, 2)).shape)
#         print(type(target))
#         img_arr = img.numpy()[0]
#         print(img_arr.shape)
#         img = Image.fromarray(img_arr)
#         plt.imshow(img)
#         plt.axis('off')  # 不显示坐标轴
#         plt.show()
#         img_path = train_label['path'][i]
#         image = Image.open(img_path)
#         plt.imshow(image)
#         plt.axis('off')  # 不显示坐标轴
#         plt.show()
#         break

In [31]:
print(1e-3)
#, weight_decay=1e-3

0.001


In [ ]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.AdamW(model.parameters(), 0.005)
scheduler = jt.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.85)
best_acc = 0.0

acc_list = []
for epoch in range(epoch_num):
    # scheduler.step()
    print('Epoch: ', epoch)

    # train(train_loader, model, criterion, optimizer, epoch, bs_value)
    val_acc = validate(val_loader, model, criterion, bs_value)
    # if train_acc.avg.item() > best_acc:
    best_acc = round(val_acc.avg.item(), 2)
    acc_list.append(best_acc)
    model.save(f'../models_pkl/eca_layer+textfeature/model_Epoch{epoch}_{best_acc}.pkl')

In [123]:
model_path = "./model_ECA_bs4/model_Epoch49_100.0.pkl"

model = attention_classifier(in_channels=512, out_channels=374)


if os.path.exists(model_path):
    model.load_state_dict(jt.load(model_path))
    print("使用以往的模型继续训练")
else:
    print("使用hugging face预训练模型")



test_pred = predict(test_label, model, 1)
print(test_pred.shape)

使用以往的模型继续训练


100%|██████████| 3073/3073 [03:06<00:00, 16.51it/s]

(3073, 374)


In [33]:
test_pred = predict(test_label, model, 1)
print(test_pred.shape)

100%|██████████| 3073/3073 [03:12<00:00, 15.96it/s]

(3073, 374)


In [34]:
top_k = 5
top_k_indices = np.argsort(test_pred, axis=1)[:, -top_k:]
top_k_indices = np.flip(top_k_indices, axis=1)

In [35]:
test_label[['Top1', 'Top2', 'Top3', 'Top4', 'Top5']] = top_k_indices

In [ ]:
test_label[70: 100]

In [44]:
columns_to_save = ['img_name', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5']
df_selected = test_label[columns_to_save]
df_selected.to_csv('./model_output/result.txt', sep=' ', index=False, header=False)